In [91]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

# Open browser

In [92]:
# 크롬드라이버 실행. 4.6 버전부터 크롬드라이버 별도 다운로드 및 경로 설정 불필요
driver = webdriver.Chrome() 

#크롬 드라이버에 url 주소 넣고 실행
site = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'
driver.get(site)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# Get community review list

In [93]:
# Open community review window by clicking Show more reviews
path = "//button[@tabindex='0']"
#path = "//span[text()='Show more reviews']" # successful
#path = '//*[@id="all_reviews"]/div[3]/div[1]/button' # NoSuchElementException

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Show reviews by recent
path = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

driver.find_element(By.XPATH, path).click()
time.sleep(1)

# Get reviews

In [94]:
class_review = "communityReview__reviewText--2bfLj"
class_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"
class_date = class_date.replace(' ', '.')
   

max_rev = 100
max_scr = 5
time_scr = 2

def extract_text(class_name, driver=driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


reviews = list()
dates = list()
n_scr = 0

while True:
    list_r = extract_text(class_review)
    list_d = extract_text(class_date)

    n = len(list_r) - len(reviews)
    if n > 0: # get new n reviews
        reviews.extend(list_r[-n:])
        dates.extend(list_d[-n:])
        n_scr = 0
    else:
        n_scr += 1

    if ((len(reviews) > max_rev) or (n_scr > max_scr)):
        break
    else:
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(time_scr)

#len(reviews), len(dates)

# close browser
driver.close()

# Save reviews

In [95]:
import pandas as pd

df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
df_reviews['date'] = pd.to_datetime(df_reviews['date'])
df_reviews.head()

,date,review
0,2024-01-24,Little too cherry on the front end for me
1,2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m..."
2,2024-01-19,Aight
3,2024-01-18,무난한 맛 가성비 좋은듯
4,2024-01-17,oak cherry black cherry chocolate blackcurrant...


In [84]:
f = 'wine_reviews'
df_reviews.to_csv(f, index=False)  

# Module

In [202]:
# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 키를 조작하기 위한 import
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# 페이지 로딩을 기다리는데에 사용할 time 모듈 import
import time

import pandas as pd 
import numpy as np
from tqdm import tqdm


In [199]:
locator_more = "//button[@tabindex='0']"
locator_recent = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

locator_review = "communityReview__reviewText--2bfLj"
locator_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"


def extract_text(class_name, driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


def vivino_reviews(wine_url, wine_name, 
                   max_rev = 100, 
                   max_scr = 10, # max num of scroll for new review list 
                   time_scr = 3, # time to wait after scrolling. StaleElementReferenceException if not enough
                   loc1=locator_more, 
                   loc2=locator_recent, 
                   loc3=locator_review, 
                   loc4=locator_date,
                   # take the final review list as the review page has same old reiviews at the end every update by scroll-down 
                   final_only = True
                  ):
    driver = webdriver.Chrome() 
    driver.get(wine_url)
    time.sleep(3)

    # enter community reviews
    driver.find_element(By.XPATH, loc1).click()
    time.sleep(1)
    
    # Show reviews by recent
    driver.find_element(By.XPATH, loc2).click()
    time.sleep(1)

    # collect reviews
    loc3 = loc3.replace(' ', '.')
    loc4 = loc4.replace(' ', '.')
     
    reviews = list()
    dates = list()
    n_scr = 0
    
    pbar = tqdm(total=max_rev)
    
    while True:
        list_r = extract_text(loc3, driver)
        list_d = extract_text(loc4, driver)
    
        n = len(list_r) - len(reviews)
        if n > 0: # get new n reviews
            if final_only: # replace review list with the latest scan
                reviews = list_r
                dates = list_d
            else: # update review list with new reviews from the latest scan
                reviews.extend(list_r[-n:])
                dates.extend(list_d[-n:])
            n_scr = 0
            pbar.update(n)
        else:
            n_scr += 1
    
        if ((len(reviews) > max_rev) or (n_scr > max_scr)):
            #print('testing:', n_scr)
            break
        else:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(time_scr)

    pbar.close()

    # close browser
    driver.close()

    # save result
    df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])
    
    if False: #deprecated
        df_reviews.to_csv(filename, index=False)  

    return df_reviews


def generate_wine_id(df_reviews):
    if df_reviews.empty:
        wid = 0
    else:
        wid = df_reviews.id.max()
        wid = 0 if wid is np.nan else wid+1 # check if no data row
    return wid

In [200]:
kwargs = {
    'max_rev': 100, 'max_scr': 10, 'time_scr': 3
}

df_reviews = pd.DataFrame()
col_rev = ['id', 'wine', 'date', 'review']

In [203]:
%%time

wine_name = 'Casillero del Diablo Cabernet Sauvignon (Reserva)'
wine_url = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'

df = vivino_reviews(wine_url, wine_name, **kwargs)

wine_id = generate_wine_id(df_reviews)
df[['id', 'wine']] = [wine_id, wine_name]
df = df.reindex(columns=col_rev)
df.to_csv(f'wine_{wine_id}.csv', index=False)

df_reviews = pd.concat([df_reviews if not df_reviews.empty else None, df])

 73%|█████████████████████████████████████████████▎                | 73/100 [01:00<00:28,  1.07s/it]

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.224); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
#0 0x55818ad22f83 <unknown>
#1 0x55818a9dbcf7 <unknown>
#2 0x55818a9e22a3 <unknown>
#3 0x55818a9e4798 <unknown>
#4 0x55818a9e48ec <unknown>
#5 0x55818aa252e2 <unknown>
#6 0x55818aa550b2 <unknown>
#7 0x55818aa1eb49 <unknown>
#8 0x55818aa554ce <unknown>
#9 0x55818aa74006 <unknown>
#10 0x55818aa54e53 <unknown>
#11 0x55818aa1cdd4 <unknown>
#12 0x55818aa1e1de <unknown>
#13 0x55818ace7531 <unknown>
#14 0x55818aceb455 <unknown>
#15 0x55818acd3f55 <unknown>
#16 0x55818acec0ef <unknown>
#17 0x55818acb799f <unknown>
#18 0x55818ad10008 <unknown>
#19 0x55818ad101d7 <unknown>
#20 0x55818ad22124 <unknown>
#21 0x7fe101a94ac3 <unknown>


In [143]:
wine_name = 'Yellow Tail Cabernet Sauvignon'
wine_url = 'https://www.vivino.com/yellow-tail-cabernet-sauvignon/w/2549?ref=nav-search#all_reviews'

wine_id = generate_wine_id(df_reviews)
kwargs.update({'filename':f'wine_{wine_id}.csv'})

df = vivino_reviews(wine_url, wine_name, **kwargs)

df[['id', 'wine']] = [wine_id, wine_name]
df_reviews = pd.concat([df_reviews if not df_reviews.empty else None, df])

In [191]:
%%time

print(1)

1
CPU times: user 46 µs, sys: 4 µs, total: 50 µs
Wall time: 53.4 µs


# lab

In [162]:
locator_more = "//button[@tabindex='0']"
locator_recent = '//*[@data-testid="baseModalBackdrop"]/div[2]/div[2]/div[2]/div/a[1]'

locator_review = "communityReview__reviewText--2bfLj"
locator_date = "anchor_anchor__m8Qi- reviewAnchor__anchor--2NKFw reviewDate__reviewDate--49vpM undefined"


def extract_text(class_name, driver, by=By.CLASS_NAME):
    l = driver.find_elements(by, class_name)
    return [x.text for x in l]


def vivino_reviews(wine_url, wine_name, fname=None,
                   max_rev = 100, max_scr = 5, time_scr = 2,
                   loc1=locator_more, 
                   loc2=locator_recent, 
                   loc3=locator_review, 
                   loc4=locator_date,
                   # take the final review list as the review page has same old reiviews at the end every update by scroll-down 
                   final_only = True
                  ):
    driver = webdriver.Chrome() 
    driver.get(wine_url)
    time.sleep(3)

    # enter community reviews
    driver.find_element(By.XPATH, loc1).click()
    time.sleep(1)
    
    # Show reviews by recent
    driver.find_element(By.XPATH, loc2).click()
    time.sleep(1)

    # collect reviews
    loc3 = loc3.replace(' ', '.')
    loc4 = loc4.replace(' ', '.')
     
    reviews = list()
    dates = list()
    n_scr = 0
    
    while True:
        list_r = extract_text(loc3, driver)
        list_d = extract_text(loc4, driver)
    
        n = len(list_r) - len(reviews)
        if n > 0: # get new n reviews
            if final_only: # replace review list with the latest scan
                reviews = list_r
                dates = list_d
            else: # update review list with new reviews from the latest scan
                reviews.extend(list_r[-n:])
                dates.extend(list_d[-n:])
            n_scr = 0
        else:
            n_scr += 1
    
        if ((len(reviews) > max_rev) or (n_scr > max_scr)):
            break
        else:
            driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
            time.sleep(time_scr)

    # close browser
    driver.close()

    # save result
    df_reviews = pd.DataFrame.from_dict({'date':dates, 'review':reviews})
    df_reviews['date'] = pd.to_datetime(df_reviews['date'])
    
    if fname is not None:
        df_reviews.to_csv(fname, index=False)  

    return df_reviews, list_r, list_d

In [163]:
kwargs = {
    'max_rev': 20, 'max_scr': 5, 'time_scr': 2
}

df_reviews = pd.DataFrame(columns=['id', 'wine', 'date', 'review'])



wine_name = 'Casillero del Diablo Cabernet Sauvignon (Reserva)'
wine_url = 'https://www.vivino.com/casillero-del-diablo-cabernet-sauvignon-reserva-central-valley/w/1135067?ref=nav-search#all_reviews'

df, reviews, dates = vivino_reviews(wine_url, wine_name, **kwargs)

wine_id = generate_wine_id(df_reviews)
df[['id', 'wine']] = [wine_id, wine_name]
df_reviews = pd.concat([df_reviews if not df_reviews.empty else None, df])

In [164]:
df_reviews.loc[df_reviews.duplicated()]

,date,review,id,wine


In [165]:
df_reviews

,date,review,id,wine
0,2024-01-24,Little too cherry on the front end for me,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
1,2024-01-20,"En liten skarp knekk i smaken. Ok fredagsvin,m...",0,Casillero del Diablo Cabernet Sauvignon (Reserva)
2,2024-01-19,Aight,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
3,2024-01-18,무난한 맛 가성비 좋은듯,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
4,2024-01-17,oak cherry black cherry chocolate blackcurrant...,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
5,2024-01-17,"Muito bom, combina muito bem com carnes vermel...",0,Casillero del Diablo Cabernet Sauvignon (Reserva)
6,2024-01-16,Så jävla sött,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
7,2024-01-12,"Ready to drink, fresh, zingy Cab Sauv. Red fru...",0,Casillero del Diablo Cabernet Sauvignon (Reserva)
8,2024-01-08,가성비 훌륭함,0,Casillero del Diablo Cabernet Sauvignon (Reserva)
9,2024-01-08,"Kanse stått uppe för länge, så lite för besk.🍇",0,Casillero del Diablo Cabernet Sauvignon (Reserva)


In [157]:
len(reviews)

33

In [158]:
dates

['Jan 24, 2024',
 'Jan 20, 2024',
 'Jan 19, 2024',
 'Jan 18, 2024',
 'Jan 17, 2024',
 'Jan 17, 2024',
 'Jan 16, 2024',
 'Jan 12, 2024',
 'Jan 8, 2024',
 'Jan 8, 2024',
 'Dec 25, 2023',
 'Dec 25, 2023',
 'Dec 25, 2023',
 'Dec 21, 2023',
 'Dec 18, 2023',
 'Dec 15, 2023',
 'Dec 13, 2023',
 'Dec 11, 2023',
 'Dec 10, 2023',
 'Dec 10, 2023',
 'Dec 8, 2023',
 'Dec 6, 2023',
 'Dec 4, 2023',
 'Dec 3, 2023',
 'Dec 3, 2023',
 'Dec 2, 2023',
 'Nov 29, 2023',
 'Nov 28, 2023',
 'Nov 25, 2023',
 'Nov 23, 2023',
 'Aug 29, 2020',
 'May 29, 2017',
 'Mar 19, 2021']